# WFI Reference File Pipeline Example

This notebook shows how to use the WFI Reference File Pipeline to create several types of reference files to submit to CRDS. At this stage, it is in the testing phase, and this notebook is used to assemble the steps of the pipeline to make test data.

Presently, the input files come from DCL FITS files. These data are located on central storage at:

```
/grp/roman/DCL_DATA/
```

## Some imports...

In [1]:
from wfi_reference_pipeline.utilities import logging_functions

LOG_FILE = logging_functions.configure_logging('wfi_refpipe_example')

from wfi_reference_pipeline.utilities import basic_calibration as bcal
from wfi_reference_pipeline.dark import dark
from wfi_reference_pipeline.flat import flat

import asdf
import glob
import logging
import matplotlib.pyplot as plt
import os

# Set the directory of the DCL data.
dcl_dir = os.path.join('/', 'grp', 'roman', 'DCL_DATA', '20663', 'rft1-cold_run_402')
logging.info(f'Using directory {dcl_dir} for DCL data.')

## Dark File

To make a dark reference file, we need to row-subtract the reference pixels, subtract the zero frame, and ramp fit the data. We'll use several darks and combine them to reduce the noise in our dark reference file.

Reading the files is fast, but sequentially doing the ramp fitting is a little slow. We'll speed this up by spreading them to multiple cores.

In [ ]:
# Grab the files.
dark_files = glob.glob(os.path.join(dcl_dir, '*darks_skipfr_0_12_20663_00?.fits'))

# Record file names in log.
logging.info('Files used for dark creation:')
for df in dark_files:
    logging.info(f'\t{os.path.basename(df)}')

# Compute the ramp image for each dark.
# This takes a bit because we're working on
# several big files.
dark_ramps = list()
for df in dark_files:
    logging.info(f'Working on file {os.path.basename(df)}...')
    dark_cal = bcal.CalDCL(df)
    dark_cal.subtract_reference_pixels()
    dark_cal.subtract_zero_read()
    dark_cal.ramp_fit()
    dark_ramps.append(dark_cal.ramp_image)

    # Clean up since we don't need all of the dark
    # FITS files open in memory anymore.
    del dark_cal

We now have ramp images of each dark in units of ADU / second. We can pass this list of ramp images to the Dark class to make ASDF dark reference files for Roman.

In [ ]:
logging.info('')
logging.info('DARK FILE CREATION')
logging.info('')

dark_ref = dark.Dark(dark_ramps, 'dark.yaml', outroot='roman_dcl20663', clobber=True)
dark_ref.compute_dark_rate()

# Let's make a dark file the matches the HLIS MA table.
dark_ref.make_dark(n_reads=8, n_resultants=5, ma_table_name='HLIS')